In [19]:
# Example 10-2. vector_v1.py: derived from vector2d_v1.py
from array import array
import reprlib
import math

class Vector:
    typecode = 'd'
    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1>
    def __iter__(self):
        return iter(self._components)  # <2>
    def __repr__(self):
        components = reprlib.repr(self._components)  # <3>
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # <5>
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # <6> !!! not: self._components ???
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # <7>
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        return self._components[index]

v1 = Vector([3, 4, 5])
print("len(v1) : ", len(v1))
print("v1[0], v1[-1]: ", v1[0], ",", v1[-1])
v7 = Vector(range(7))
print("v7 : ", v7)
print("v7[1:4] : ", v7[1:4])

len(v1) :  3
v1[0], v1[-1]:  3.0 , 5.0
v7 :  (0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)
v7[1:4] :  array('d', [1.0, 2.0, 3.0])


In [30]:
# Example 10-4. Checking out the behavior of __getitem__ and slices.
class MySeq:
    def __getitem__(self, index):
        return index #
s = MySeq()
print("s[1] =", s[1])
print("s[1:4] =", s[1:4])
print("s[1:4:2] =", s[1:4:2]) # slice(1, 4, 2) means start at 1, stop at 4, step by 2.
print("s[1:4:2, 9] =", s[1:4:2, 9]) #  the presence of commas inside the [] means __getitem__ receives a tuple.
print("s[1:4:2, 7:9] =", s[1:4:2, 7:9])

s[1] = 1
s[1:4] = slice(1, 4, None)
s[1:4:2] = slice(1, 4, 2)
s[1:4:2, 9] = (slice(1, 4, 2), 9)
s[1:4:2, 7:9] = (slice(1, 4, 2), slice(7, 9, None))


In [36]:
# indices
print("slice(None, 10, 2).indices(5) : ", slice(None, 10, 2).indices(5))
print("slice(-3, None, None).indices(5) : ", slice(-3, None, None).indices(5))
help(slice.indices)

slice(None, 10, 2).indices(5) :  (0, 5, 2)
slice(-3, None, None).indices(5) :  (2, 5, 1)
Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.



In [66]:
# # Example 10-?
from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1>
    def __iter__(self):
        return iter(self._components)  # <2>
    def __repr__(self):
        components = reprlib.repr(self._components)  # <3>
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # <5>
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # <6> !!! not: self._components ???
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # <7>
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index]) # invoke the class to build another Vector instance ...
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

v7 = Vector(range(7))
print("v7 : ", v7)
print("v7[-1] : ", v7[-1])
print("v7[1:4] : ", repr(v7[1:4]))
print("v7[-1:] : ", repr(v7[-1:]))
print("v7[1,2] : ", repr(v7[1,2]))

v7 :  (0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)
v7[-1] :  6.0
v7[1:4] :  Vector([1.0, 2.0, 3.0])
v7[-1:] :  Vector([6.0])


TypeError: Vector indices must be integers

In [67]:
# Example 10-9.  __getattr__
v = Vector(range(5))
print("v : ", repr(v))
print("v.x : ", v.x)
v.x = 10
print("v.x : ", v.x)
print("v : ", repr(v)) # !!!

v :  Vector([0.0, 1.0, 2.0, 3.0, 4.0])
v.x :  0.0
v.x :  10
v :  Vector([0.0, 1.0, 2.0, 3.0, 4.0])


In [73]:
# Example 10-10. Part of vector_v3.py: __setattr__ method in Vector cl
from array import array
import reprlib
import math
import numbers

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1>
    def __iter__(self):
        return iter(self._components)  # <2>
    def __repr__(self):
        components = reprlib.repr(self._components)  # <3>
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # <5>
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # <6> !!! not: self._components ???
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # <7>
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index]) # invoke the class to build another Vector instance ...
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

v = Vector(range(5))
print("v : ", repr(v))
print("v.x : ", v.x)
v.x = 10

v :  Vector([0.0, 1.0, 2.0, 3.0, 4.0])
v.x :  0.0


AttributeError: readonly attribute 'x'

In [89]:
# functools.reduce
print("2 * 3 * 4 * 5 = ", 2 * 3 * 4 * 5) # the result we want: 5! == 120
import functools
print("functools.reduce(lambda a,b: a*b, range(2, 6)) : ", functools.reduce(lambda a,b: a*b, range(2, 6)))
print()

n = 0
for i in range(1, 6): #
    n ^= i
print("n = ", n)

import functools
print("functools.reduce(lambda a, b: a^b, range(6)) : ", functools.reduce(lambda a, b: a^b, range(6))) #

import operator
print("functools.reduce(operator.xor, range(6)) : ", functools.reduce(operator.xor, range(6))) #

2 * 3 * 4 * 5 =  120
functools.reduce(lambda a,b: a*b, range(2, 6)) :  120

n =  1
functools.reduce(lambda a, b: a^b, range(6)) :  1
functools.reduce(operator.xor, range(6)) :  1


In [93]:
# Example 10-12 - 14. Part of vector_v4.py: two imports and __hash__ method added to Vector class from vector_v3.py.
from array import array
import reprlib
import math
import functools #
import operator #

class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'
    def __init__(self, components):
        self._components = array(self.typecode, components)  # <1>
    def __iter__(self):
        return iter(self._components)  # <2>
    def __repr__(self):
        components = reprlib.repr(self._components)  # <3>
        components = components[components.find('['):-1]  # <4>
        return 'Vector({})'.format(components)
    def __str__(self):
        return str(tuple(self))
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))  # <5>
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))  # <6> !!! not: self._components ???
    def __bool__(self):
        return bool(abs(self))
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)  # <7>
    def __len__(self):
        return len(self._components)
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index]) # invoke the class to build another Vector instance ...
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))
    def __hash__(self):
        hashes = (hash(x) for x in self._components) #
        return functools.reduce(operator.xor, hashes, 0) #

In [100]:
# zip
print("zip(range(3), 'ABC') : ", zip(range(3), 'ABC'))
print("list(zip(range(3), 'ABC')) : ", list(zip(range(3), 'ABC')))
print("list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3])) : ", list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3])))
from itertools import zip_longest
print("list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1)) : \n    ", 
      list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1)))
# n optional fillvalue (None by default) to complete missing values 
# so it can generate tuples until the last iterable is exhausted.

zip(range(3), 'ABC') :  <zip object at 0x7f573846dac8>
list(zip(range(3), 'ABC')) :  [(0, 'A'), (1, 'B'), (2, 'C')]
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3])) :  [(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1)) : 
     [(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]


In [106]:
# Example 10-16 is a full listing introducing custom formatting.
# BEGIN VECTOR_V5
"""
A multi-dimensional ``Vector`` class, take 5
A ``Vector`` is built from an iterable of numbers::
    >>> Vector([3.1, 4.2])
    Vector([3.1, 4.2])
    >>> Vector((3, 4, 5))
    Vector([3.0, 4.0, 5.0])
    >>> Vector(range(10))
    Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])
    
Tests with 2-dimensions (same results as ``vector2d_v1.py``)::
    >>> v1 = Vector([3, 4])
    >>> x, y = v1
    >>> x, y
    (3.0, 4.0)
    >>> v1
    Vector([3.0, 4.0])
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0)
    >>> octets = bytes(v1)
    >>> octets
    b'd\\x00\\x00\\x00\\x00\\x00\\x00\\x08@\\x00\\x00\\x00\\x00\\x00\\x00\\x10@'
    >>> abs(v1)
    5.0
    >>> bool(v1), bool(Vector([0, 0]))
    (True, False)
    
Test of ``.frombytes()`` class method::
    >>> v1_clone = Vector.frombytes(bytes(v1))
    >>> v1_clone
    Vector([3.0, 4.0])
    >>> v1 == v1_clone
    True
    
Tests with 3-dimensions::
    >>> v1 = Vector([3, 4, 5])
    >>> x, y, z = v1
    >>> x, y, z
    (3.0, 4.0, 5.0)
    >>> v1
    Vector([3.0, 4.0, 5.0])
    >>> v1_clone = eval(repr(v1))
    >>> v1 == v1_clone
    True
    >>> print(v1)
    (3.0, 4.0, 5.0)
    >>> abs(v1)  # doctest:+ELLIPSIS
    7.071067811...
    >>> bool(v1), bool(Vector([0, 0, 0]))
    (True, False)
    
Tests with many dimensions::
    >>> v7 = Vector(range(7))
    >>> v7
    Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])
    >>> abs(v7)  # doctest:+ELLIPSIS
    9.53939201...
    
Test of ``.__bytes__`` and ``.frombytes()`` methods::
    >>> v1 = Vector([3, 4, 5])
    >>> v1_clone = Vector.frombytes(bytes(v1))
    >>> v1_clone
    Vector([3.0, 4.0, 5.0])
    >>> v1 == v1_clone
    True
    
Tests of sequence behavior::
    >>> v1 = Vector([3, 4, 5])
    >>> len(v1)
    3
    >>> v1[0], v1[len(v1)-1], v1[-1]
    (3.0, 5.0, 5.0)
    
Test of slicing::
    >>> v7 = Vector(range(7))
    >>> v7[-1]
    6.0
    >>> v7[1:4]
    Vector([1.0, 2.0, 3.0])
    >>> v7[-1:]
    Vector([6.0])
    >>> v7[1,2]
    Traceback (most recent call last):
      ...
    TypeError: Vector indices must be integers
    
Tests of dynamic attribute access::
    >>> v7 = Vector(range(10))
    >>> v7.x
    0.0
    >>> v7.y, v7.z, v7.t
    (1.0, 2.0, 3.0)
    
Dynamic attribute lookup failures::
    >>> v7.k
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 'k'
    >>> v3 = Vector(range(3))
    >>> v3.t
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 't'
    >>> v3.spam
    Traceback (most recent call last):
      ...
    AttributeError: 'Vector' object has no attribute 'spam'
    
Tests of hashing::
    >>> v1 = Vector([3, 4])
    >>> v2 = Vector([3.1, 4.2])
    >>> v3 = Vector([3, 4, 5])
    >>> v6 = Vector(range(6))
    >>> hash(v1), hash(v3), hash(v6)
    (7, 2, 1)

Most hash values of non-integers vary from a 32-bit to 64-bit CPython build::
    >>> import sys
    >>> hash(v2) == (384307168202284039 if sys.maxsize > 2**32 else 357915986)
    True
    
Tests of ``format()`` with Cartesian coordinates in 2D::
    >>> v1 = Vector([3, 4])
    >>> format(v1)
    '(3.0, 4.0)'
    >>> format(v1, '.2f')
    '(3.00, 4.00)'
    >>> format(v1, '.3e')
    '(3.000e+00, 4.000e+00)'
    
Tests of ``format()`` with Cartesian coordinates in 3D and 7D::
    >>> v3 = Vector([3, 4, 5])
    >>> format(v3)
    '(3.0, 4.0, 5.0)'
    >>> format(Vector(range(7)))
    '(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'

Tests of ``format()`` with spherical coordinates in 2D, 3D and 4D::
    >>> format(Vector([1, 1]), 'h')  # doctest:+ELLIPSIS
    '<1.414213..., 0.785398...>'
    >>> format(Vector([1, 1]), '.3eh')
    '<1.414e+00, 7.854e-01>'
    >>> format(Vector([1, 1]), '0.5fh')
    '<1.41421, 0.78540>'
    >>> format(Vector([1, 1, 1]), 'h')  # doctest:+ELLIPSIS
    '<1.73205..., 0.95531..., 0.78539...>'
    >>> format(Vector([2, 2, 2]), '.3eh')
    '<3.464e+00, 9.553e-01, 7.854e-01>'
    >>> format(Vector([0, 0, 0]), '0.5fh')
    '<0.00000, 0.00000, 0.00000>'
    >>> format(Vector([-1, -1, -1, -1]), 'h')  # doctest:+ELLIPSIS
    '<2.0, 2.09439..., 2.18627..., 3.92699...>'
    >>> format(Vector([2, 2, 2, 2]), '.3eh')
    '<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'
    >>> format(Vector([0, 1, 0, 0]), '0.5fh')
    '<1.00000, 1.57080, 0.00000, 0.00000>'
"""

from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools  # <1>


class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):  # <2>
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):  # <3>
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())  # <4>
            outer_fmt = '<{}>'  # <5>
        else:
            coords = self
            outer_fmt = '({})'  # <6>
        components = (format(c, fmt_spec) for c in coords)  # <7>
        return outer_fmt.format(', '.join(components))  # <8>

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
# END VECTOR_V5

if __name__ == "__main__":
    import doctest
    doctest.testmod() 

In [28]:
# The search for a Pythonic sum
my_list = [[1, 2, 3], [40, 50, 60], [9, 8, 7]]
import functools
print("functools.reduce(lambda a, b: a+b, [sub[1] for sub in my_list]) : ", 
     functools.reduce(lambda a, b: a+b, [sub[1] for sub in my_list]))
print("functools.reduce(lambda a, b: a + b[1], my_list, 0) : ", 
      functools.reduce(lambda a, b: a + b[1], my_list, 0))

import numpy as np
my_array = np.array(my_list)
print("np.sum(my_array[:, 1]) : ", np.sum(my_array[:, 1]))
print("!!! my_array[:, 1] : ", my_array[:, 1])

import operator
print("functools.reduce(operator.add, [sub[1] for sub in my_list], 0) : ",
     functools.reduce(operator.add, [sub[1] for sub in my_list], 0))

print("sum([sub[1] for sub in my_list]) : ", sum([sub[1] for sub in my_list]))
print()

print("!!! most Pythonic - sum(sub[1] for sub in my_list) : ", sum(sub[1] for sub in my_list))

functools.reduce(lambda a, b: a+b, [sub[1] for sub in my_list]) :  60
functools.reduce(lambda a, b: a + b[1], my_list, 0) :  60
np.sum(my_array[:, 1]) :  60
!!! my_array[:, 1] :  [ 2 50  8]
functools.reduce(operator.add, [sub[1] for sub in my_list], 0) :  60
sum([sub[1] for sub in my_list]) :  60

!!! most Pythonic - sum(sub[1] for sub in my_list) :  60
